In [ ]:
# 아래 코드는 Tensorflow 공식 문서를 참고하여 일부를 수정한 코드입니다.
# https://www.tensorflow.org/text/tutorials/transformer


# Transformer는 self attention에서 모든 위치의 단어가 한번에 입력으로 들어갑니다.
# 이 때 각 단어의 위치 정보를 함께 나타내기 위해 Positional Embedding이라는 장치를 통해
# 각 단어를 새롭게 임베딩 합니다.
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model, input_length):
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = self.positional_encoding(length=input_length, depth=d_model)

    def positional_encoding(self, length, depth):
        depth = depth / 2

        positions = np.arange(length)[:, np.newaxis]       # (seq, 1)
        depths = np.arange(depth)[np.newaxis, :] / depth   # (1, depth)

        angle_rates = 1 / (10000 ** depths)       # (1, depth)
        angle_rads = positions * angle_rates      # (pos, depth)

        pos_encoding = np.concatenate(
            [np.sin(angle_rads), np.cos(angle_rads)],
            axis=-1
        )

        return tf.cast(pos_encoding, dtype=tf.float32)

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)

        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]

        return x


# 하나의 Transformer 인코더는 여러개의 인코더 Layer가 일렬로 연결된 구조입니다.
# 이 클래스는 그 하나의 Layer를 의미합니다.
# 여기서 Transformer의 핵심인 Muti Head Attention이 계산됩니다.
class TransformerEncoderLayer(tf.keras.Model):
    def __init__(
        self,
        d_model,
        num_heads,
        d_ff,
        dropout_rate
    ):
        super(TransformerEncoderLayer, self).__init__()

        # Attention
        self.mha = tf.keras.layers.MultiHeadAttention(
            key_dim=d_model,
            num_heads=num_heads,
            dropout=dropout_rate
        )
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

        # Feed-Forward
        self.ff = tf.keras.Sequential([
            tf.keras.layers.Dense(d_ff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.ff_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        # Attention
        attn_out = self.mha(x, x)
        attn_out = self.dropout(attn_out)
        out = x + attn_out
        out = self.layer_norm(out)

        # Feed-Forward
        ff_out = self.ff(out)
        out = out + ff_out
        out = self.ff_norm(out)

        return out


# Transformer 인코더 layer를 복수로 쌓을 수 있게 만든 모델 클래스입니다.
# 이 모델에 Positional Embedding과 Classifier를 함께 추가하여 분류 모델로 사용합니다.
class TransformerEncoder(tf.keras.Model):
    def __init__(
        self,
        num_layers,
        d_model,
        num_heads,
        d_ff,
        vocab_size,
        num_classes,
        input_length,
        dropout_rate=0.0,
    ):
        super(TransformerEncoder, self).__init__()

        # Embedding
        self.pos_embedding = PositionalEmbedding(vocab_size, d_model, input_length)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

        # Encoder Layers
        self.enc_layers = [
            TransformerEncoderLayer(
                d_model=d_model,
                num_heads=num_heads,
                d_ff=d_ff,
                dropout_rate=dropout_rate
            )
            for _ in range(num_layers)
        ]

        # Classifier
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    def call(self, x):
        # Embedding
        out = self.pos_embedding(x)
        out = self.dropout(out)

        # Encoder Layer
        for enc_layer in self.enc_layers:
            out = enc_layer(out)

        # Classifier
        out = self.classifier(out)

        return out

In [ ]:
transformer_model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    tokenizer_layer,
    TransformerEncoder(
        num_layers=2, # 인코더 layer는 두개 사용
        d_model=embedding_dim, # 임베딩된 단어 벡터의 차원
        num_heads=4, # Multi Head Attention에서 head의 개수
        d_ff=256, # Feed-Forward layer에서 사용할 차원 수
        vocab_size=tokenizer_layer.vocabulary_size(), # Tokenizer가 찾은 전체 토큰 개수
        num_classes=num_classes, # 클래스 개수
        input_length=max_sequence_length, # 입력 문장의 최대 토큰 수. 앞서 50개로 지정했었습니다.
        dropout_rate=0.05
    )
])

# 모델을 구성하는 layer들의 요약 정보를 확인합니다.
transformer_model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
num_epochs = 5
transformer_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = transformer_model.fit(
    X_train,
    y_train,
    epochs=num_epochs,
    batch_size=64,
    shuffle=True,
    validation_data=(X_valid, y_valid)
)

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()